<a href="https://colab.research.google.com/github/huiesoo/esaa/blob/main/24-1/0506.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 09 추천 시스템

## 01 추천 시스템의 개요와 배경

### 추천 시스템의 개요

### 온라인 스토어의 필수 요소, 추천 시스템

### 추천 시스템의 유형



*   콘텐츠 기반 필터링(Content based filtering)
*   협업 필터링(Collaborate Filtering)
    *   최근접 이웃(Nearest Neighbor)
    *   잠재 요인(Latent Factor)



## 02 콘텐츠 기반 필터링 추천 시스템

## 03 최근접 이웃 협업 필터링



*   협업 필터링
    *   사용자 행동 양식(User Behavior)만을 기반으로 추천을 수행하는 것
    *   사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가(Predicted Rating)
*   사용자-아이템 평점 행렬
    *   행: 개별 사용자, 열: 개별 아이템
    *   사용자 아이디 행, 아이템 아이디 열 위치에 해당하는 값이 평점을 나타내는 형태
    *   다차원 행렬
    *   희소 행렬(Sparse MAtrix)
*   최근접 이웃 협업 필터링
    *   사용자 기반(User-User): 당신과 비슷한 고객들이 다음 상품도 구매했습니다(Customers like you also bought these items)
        *   특정 사용자와 타 사용자 간의 유사도를 측정한 뒤 가장 유사도가 높은 TOP-N 사용자를 추출해 그들이 선호하는 아이템을 추천
    *   아이템 기반(Item-Item): 이 상품을 선택한 다른 고객들은 다음 상품도 구매했습니다(Customers who bought this item also bought these items)
        *   아이템이 가지는 속성과는 상관없이 사용자들이 그 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템을 추천하는 기준이 되는 알고리즘
    *   사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 더 높음 <- 비슷한 영화를 좋아한다고 해서 사람들의 취향이 비슷하다고 판단하기는 어려운 경우가 많기 때문



## 04 잠재 요인 협업 필터링



*   사용자-아이템 평점 매트릭스 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법



### 행렬 분해의 이해



*   다차원의 매트릭스를 저차원 매트릭스로 분해하는 기법
*   SVD(Singular Vetor Decomposition), NMP(Non-Negative Matrix Factorization) 등



### 확률적 경사 하강법을 이용한 행렬 분해



*   P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것



In [3]:
import numpy as np

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K=3

np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [5]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
  error = 0

  full_pred_matrix = np.dot(P, Q.T)

  x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)

  return rmse

In [6]:
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

for step in range(steps):
  for i, j, r in non_zeros:
    eij = r - np.dot(P[i, :], Q[j, :].T)
    P[i, :] = P[i, :] + learning_rate*(eij*Q[j, :] - r_lambda*P[i, :])
    Q[j, :] = Q[j, :] + learning_rate*(eij*P[i, :] - r_lambda*Q[j, :])
    rmse = get_rmse(R, P, Q, non_zeros)
    if(step % 50) == 0:
      print("### iteration step : ", step, "rmse : ", rmse)

### iteration step :  0 rmse :  3.261355059488935
### iteration step :  0 rmse :  3.26040057174686
### iteration step :  0 rmse :  3.253984404542389
### iteration step :  0 rmse :  3.2521583839863624
### iteration step :  0 rmse :  3.252335303789125
### iteration step :  0 rmse :  3.251072196430487
### iteration step :  0 rmse :  3.2492449982564864
### iteration step :  0 rmse :  3.247416477570409
### iteration step :  0 rmse :  3.241926055455223
### iteration step :  0 rmse :  3.2400454107613084
### iteration step :  0 rmse :  3.240166740749792
### iteration step :  0 rmse :  3.2388050277987723
### iteration step :  50 rmse :  0.5003190892212748
### iteration step :  50 rmse :  0.5001616291326989
### iteration step :  50 rmse :  0.49899601202578087
### iteration step :  50 rmse :  0.4988483450145831
### iteration step :  50 rmse :  0.49895189256631756
### iteration step :  50 rmse :  0.49833236830090993
### iteration step :  50 rmse :  0.4984148489378701
### iteration step :  50 rmse 

In [7]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
